In [1]:
from porter import stem
from TextRepresenter import PorterStemmer
import collections
import math
import re
#Exercice1:
doc1 ="the new home has been saled on top forecasts"
doc2 ="the home sales rise in july"
doc3 = "there is an increase in home sales in july"
doc4 ="july encounter a new home sales rise"
stop_words= ["the", "a", "an", "on", "behind", "under", "there", "in", "on"]

In [2]:
def index(doc):
    doc_sep=doc.split() 
    doc_sep_norm=[stem(i.lower()) for i in doc_sep]
    index=collections.Counter(doc_sep_norm)
    print(index)

In [3]:
#2
docs=[doc1.split(), doc2.split(), doc3.split(), doc4.split()]
index = {}
index_inv = {}
for i in range(len(docs)):
    doc = docs[i]
    index_doc={}    
    for word in doc:
        if word in stop_words:
            continue
        stemmed = stem(word.lower())
        if word in index_doc:
            index_doc[stemmed]+=1
        else:
            index_doc[stemmed]=1
        
        
        if stemmed in index_inv:
            if i in index_inv[stemmed]:
                index_inv[stemmed][i]+=1
            else:
                index_inv[stemmed][i]=1
        else:
            index_inv[stemmed]={i:1}
    index[i]=index_doc
for word in index_inv:
    df=len(index_inv[word])
    idf=math.log((1+len(docs))/(1+df))
    for doc in index_inv[word]:
        index[doc][word]=index[doc][word]/len(docs[doc])*idf
        index_inv[word][doc]= index_inv[word][doc]/len(docs[doc])*idf
    
print(index)
print(index_inv)  

{0: {'new': 0.05675840264066563, 'home': 0.0, 'ha': 0.10181008131935056, 'been': 0.10181008131935056, 'sale': 0.0, 'top': 0.10181008131935056, 'forecast': 0.10181008131935056}, 1: {'home': 0.0, 'sale': 0.0, 'rise': 0.08513760396099845, 'juli': 0.037190591885701625}, 2: {'is': 0.10181008131935056, 'increas': 0.10181008131935056, 'home': 0.0, 'sale': 0.0, 'juli': 0.024793727923801082}, 3: {'juli': 0.03187765018774425, 'encount': 0.13089867598202215, 'new': 0.07297508910942724, 'home': 0.0, 'sale': 0.0, 'rise': 0.07297508910942724}}
{'new': {0: 0.05675840264066563, 3: 0.07297508910942724}, 'home': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}, 'ha': {0: 0.10181008131935056}, 'been': {0: 0.10181008131935056}, 'sale': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0}, 'top': {0: 0.10181008131935056}, 'forecast': {0: 0.10181008131935056}, 'rise': {1: 0.08513760396099845, 3: 0.07297508910942724}, 'juli': {1: 0.037190591885701625, 2: 0.024793727923801082, 3: 0.03187765018774425}, 'is': {2: 0.10181008131935056}, 'increas': 

In [4]:
#Exercice 2
def buildDocCollectionSimple(nom_doc):
    var = open(nom_doc,'r')
    resultat={}
    ok=False
    while True: 
        if ok==False:
            texte=var.readline()
       
        if texte =='':
            break
        else:
            ligne=texte.split()
            if ".I" in texte:
                id = ligne[1]
                texte= var.readline()
                chaine = ""
                if ".T" in texte:
                    while True:                    
                        texte=var.readline()
                        if texte =='' or texte.startswith('.'):
                            if ".I" in texte:
                                ok = True  
                            else:
                                ok = False
                            break                    
                        chaine=chaine+" "+texte
                else:
                    ok = False
                    
                resultat[id]=chaine
    return resultat

print(buildDocCollectionSimple("cacmShort-good.txt"))        

{'1': ' Preliminary Report-International Algebraic Language\n jgkfldjgfkld\n', '2': ' Extraction of Roots by Repeated Subtractions for Digital Computers\n', '3': ' Techniques Department on Matrix Program Schemes\n', '4': ' Glossary of Computer Engineering and Programming Terminology\n', '5': ' Two Square-Root Approximations\n', '6': ' The Use of Computers in Inspection Procedures\n', '7': ' Glossary of Computer Engineering and Programming Terminology\n', '8': ' On The Equivalence and Transformation of Program Schemes\n', '9': ' Proposal for an UNCOL\n', '10': ' Glossary of Computer Engineering and Programming Terminology\n', '11': ' The Problem of Programming Communication with\n Changing Machines A Proposed Solution-Part 2\n'}


In [6]:
def buildDocumentCollectionRegex(nom_doc):
    var = open(nom_doc,'r')
    fichier=var.read().replace("\n",'#').replace("\t",' ')
    x = re.findall("(?<=\.I )(\d*.*?)(?=(\.I \d*|\Z))",fichier)
    resultat = {}
    for elt in x:
        st = elt[0]+elt[1]
        numero=re.findall("(\d*?)#\.",elt[0])
        dictt = {}
        for key in [("title", "T"), ("text", "W"), ("author", "A"), ("keywords", "K")]:
            x = re.findall("(?<=\."+key[1]+"#)(.*?)(?=\.[A-Z]#)",st)
            if len(x)>0:
                x = re.sub("#", " ", x[0])
            else : 
                x = ""
            dictt[key[0]] = x
            
        resultat[numero[0]]= dictt
        
    return resultat

formatted = buildDocumentCollectionRegex("cacm.txt")

In [7]:
import json
with open('cacm_formatted.json', 'w') as fp:
    json.dump(formatted, fp)

In [9]:
class Document:
    def __init__(self, I, T):
        self.I=I
        self.T=T
    def get_T(self):
        res = ""
        for field in self.T:
            if len(self.T[field])>0:
                res  = res +" "+ self.T[field]
        return res
    def get_id(self):
        return self.I

class Parser:  
    def parse(self,documents):
        result = {}
        formatted = buildDocumentCollectionRegex(documents)
        return {k: Document(k, v) for k, v in formatted.items()}
            
parser = Parser()
res = parser.parse("cacm.txt")

In [161]:
print(res["1"].get_T())

 Preliminary Report-International Algebraic Language  Perlis, A. J. Samelson,K. 


In [10]:
import time
class IndexerSimple:
    def __init__(self,documents):
        self.parsed = {}
        self.index,self.index_inverse = self.indexation(documents)
        var = open("index.txt","w")
        var.write(str(self.index))
        var.close()
        var = open("index-inverse.txt","w")
        var.write(str(self.index_inverse))
        var.close()       
        
        
    def indexation(self,documents):
        parser = Parser()
        self.parsed =parser.parse(documents)
        stemmer = PorterStemmer()
        index = {}
        index_inv={}
        for doc in self.parsed:            
            index[doc]=stemmer.getTextRepresentation(self.parsed[doc].get_T())
            for stemmed in index[doc]:
                if stemmed in index_inv:
                    if doc in index_inv[stemmed]:
                        index_inv[stemmed][doc]+=1
                    else:
                        index_inv[stemmed][doc]=1
                else:
                    index_inv[stemmed]={doc:1}
        return index,index_inv
    
    def getTfsForDoc(self,doc):
        return self.index[doc]
    
    def getTfIDFsForDoc(self,doc):
        resultat={}
        N = len(self.index)
        for word in index[doc]:
            df=len(self.index_inverse[word])
            idf=math.log((1+N)/(1+df))
            resultat[word]=self.index[doc][word]/len(self.index[doc])*idf
        return resultat
    
    def getTfsForStem(self,stem):
        return self.index_inverse[stem]
    
    def getTfIDFsForStem(self,stem):
        resultat={}
        df=len(self.index_inverse[stem])
        idf=math.log((1+len(self.index))/(1+df))
        for doc in self.index_inverse[stem]:
            resultat[doc]= self.index_inverse[stem][doc]/len(self.index[doc])*idf
        return resultat
    
    def getIDFForStem(self, stem):
        if stem in self.index_inverse:
            df=len(self.index_inverse[stem])
            idf=math.log((1+len(self.index))/(1+df))     
        return idf

    
    def  getStrDoc(self,document):
        return ".I "+document+"\n.Content \n"+self.parsed[document].get_T()
    
#    def indexation2(self,docs):
#        parser = Parser()
#        parsed_col= parser.parse(docs)
#        index = {}
#        index_inv = {}
#        for i in parsed_col:
#            doc = parsed_col[i].T
#            index_doc={}    
#            for word in doc:
#                if word in stop_words:
#                    continue
#                stemmed = stem(word.lower())
#                if word in index_doc:
#                    index_doc[stemmed]+=1
#                else:
#                    index_doc[stemmed]=1
#                if stemmed in index_inv:
#                    if i in index_inv[stemmed]:
#                        index_inv[stemmed][i]+=1
#                    else:
#                        index_inv[stemmed][i]=1 
#                else:
#                    index_inv[stemmed]={i:1}
#            index[i]=index_doc
#        return index,index_inv
        
            
#indexer=IndexerSimple("cacm/cacm.txt")
#s=time.time()
#index,index_inv=indexer.indexation("cacm/cacm.txt")
#s2=time.time()
#print(s2-s)         

In [11]:
import numpy as np
#TME2:
#exercice 1:
def boolean_model(index_inverse, requete):
    docs=[]
    for word in requete:
        stemmed = stem(word.lower())
        if word not in stop_words and stemmed in index_inverse:       
            docs.append(set(index_inverse[stemmed].keys()))
    
    res = docs[0].intersection(*docs)
    return res

In [12]:
import abc
class Weighter:
    def __init__(self,index):
        self.index=index #object index
        self.words=self.WordVector()
        
    __metaclass__ = abc.ABCMeta
    

    @abc.abstractmethod
    def getWeightsForDoc(self,idDoc):
        """Method documentation"""
        return
    @abc.abstractmethod
    def getWeightsForStem(self,stem):
        """Method documentation"""
        return
    @abc.abstractmethod
    def getWeightsForQuery(self,query):
        """Method documentation"""
        return
    
class Weighter_TF_bool(Weighter):
    
    def getWeightsForDoc(self,idDoc):
        return self.index.getTfsForDoc(idDoc)
        
      
    def getWeightsForStem(self,stem):
        return self.index.getTfsForStem(stem)
     
    
    def getWeightsForQuery(self,query):
        result={}   
        for word in query.split():
            stemmed = stem(word.lower())
            if stemmed not in stop_words:
                result[stemmed]=1
        return result

class Weighter_TF_TF(Weighter):
    def getWeightsForDoc(self,idDoc):
        return self.index.getTfsForDoc(idDoc)
       

    def getWeightsForStem(self,stem):
        return self.index.getTfsForStem(stem)
     

    def getWeightsForQuery(self,query):
        stemmer = PorterStemmer()
        dict=stemmer.getTextRepresentation(query)      
        return dict
    
class Weighter_TF_IDF(Weighter):
    def getWeightsForDoc(self,idDoc):
        return self.index.getTfsForDoc(idDoc)
       

    def getWeightsForStem(self,stem):
        return self.index.getTfsForStem(stem)
    
    def getWeightsForQuery(self,query):
        result = {}
        for word in query.split():
            if word not in stop_words:
                stem = stem(word.lower())
                if stem in self.index.index_inverse:
                    df=len(self.index.index_inverse[stem])
                    idf=math.log((1+len(self.index.index))/(1+df))
                    result[stem]=idf
       
        return result 
    
class Weighter_logTf_IDF(Weighter):
    
    def getWeightsForDoc(self,idDoc):
        result = {}
        dict=self.index.getTfsForDoc(idDoc)
        for word in dict:
            result[word]= 1 + math.log(dict[word])
        return result
    
    def getWeightsForStem(self,stem):
        result={}
        dict=self.index.getTfsForStem(stem)
        for doc in dict:
            result[doc]=1+math.log(dict[doc])            
        return result
    
    def getWeightsForQuery(self,query):
        result = {}
        for word in query.split():
            if word not in stop_words:
                stem = stem(word.lower())
                if stem in self.index.index_inverse:
                    df=len(self.index.index_inverse[stem])
                    idf=math.log((1+len(self.index.index))/(1+df))
                    result[stem]=idf

        return result 

class Weighter_logTFidf_logTFidf(Weighter):
    
    def getWeightsForDoc(self,idDoc):
        result={}
        dict=self.index.getTfsForDoc(idDoc)
        for word in dict:
            df=len(self.index.index_inverse[word])
            idf=math.log((1+len(self.index.index))/(1+df))
            result[word]=(1+math.log(dict[word]))*idf
       
        return result
    
    def getWeightsForStem(self,stem):
        result={}
        df=len(self.index.index_inverse[stem])
        idf=math.log((1+len(self.index.index))/(1+df))
        dict=self.index.getTfsForStem(stem)
        for doc in dict:
            result[doc]=(1+math.log(dict[doc]))*idf         
        return result
    
    def getWeightsForQuery(self,query):
        result={}
        stemmer = PorterStemmer()
        dict=stemmer.getTextRepresentation(query)      
        for stem in dict: 
            if stem in self.index.index_inverse:
                df=len(self.index.index_inverse[stem])
                idf=math.log((1+len(self.index.index))/(1+df))
                result[stem]=(1+math.log(dict[stem]))*idf

        return result 

In [58]:
from operator import itemgetter

class IRModel:
 
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod
    def getScores(self, query):
        """Method documentation"""
        return
    
    def getRanking(self, query): 
        result=self.getScores(query)
        top=list(dict(sorted(result.items(), key = itemgetter(1), reverse = True)[:100]).keys())
        return top
    
    
class Vectoriel(IRModel):
    def __init__(self, weighter, normalised):
            self.weighter = weighter
            self.normalised = normalised
            self.normes = self.get_normes()
    def get_normes():
        index = self.weighter.index.index
        resut = {}
        for doc in index :
            norme = math.sqrt((np.array(list(index[doc].values()))**2).sum())
            result[doc] = norme
        return result
            
    def getScores(self, query):
        result={}
        weights_query = self.weighter.getWeightsForQuery(query)
        norme_query = math.sqrt((np.array(list(weights_query.values()))**2).sum())
        
        candidats=list({weighter.index.index_inverse[i.keys()] for i in weights_query})
        for doc in candidats :
            weights_doc = self.weighter.getWeightsForDoc(doc)
            intersect = np.intersect1d(list(weights_query.keys()), list(weights_doc.keys()))
            score = 0
            if normalised:
                for word in intersect:
                        score = score + weights_doc[word]*weights_query[word]
            else:
                norme_intersect = 0
                for word in intersect:
                    norme_intersect = norme_intersect + weights_doc[word]*weights_query[word]
                  
                score = norme_intersect/(self.normes[doc]+ norme_query)
            
            result[doc] = score
            
        return result
class ModeleLangue(IRModel):          
    def __init__(self,index,landa):
        self.index=index
        self.landa=landa
    def getScores(self, query):
        stemmer = PorterStemmer()
        new_query=stemmer.getTextRepresentation(query)
        result={}
        docs=self.index.index
        index_inverse=self.index.index_inverse
        tf_c_all=0
        for word in index_inverse:
            tf_c_all+=sum(index_inverse[word].values())
        for doc in docs:
            doc_words=docs[doc] #getTfsForDoc
            proba_doc=1
            for word in new_query:
                tf_c=sum(index_inverse.get(word,{0:0}).values()) #getTfsForstem
                if(tf_c==0):
                    tf_c=1
                proba_word=((1-self.landa)*(doc_words.get(word,0)/sum(doc_words.values()))) + (self.landa*(tf_c/tf_c_all))
                proba_doc= proba_doc*proba_word
            result[doc]=proba_doc
        return result
        
class BM25(IRModel):
    
    def __init__(self, index, k, b): #Objet index
        self.index = index
        self.k = k
        self.b = b
        self.lengths, self.avg_length = self.lengths()
        
    def lengths(self):
        index_=self.index.index
        res={}
        total = 0
        for doc in index_:
            length = len(index_[doc])
            res[doc] = length
            total += length
        return res, total/len(index_)
                       
    def getScores(self, query):
        stemmer = PorterStemmer()
        words = stemmer.getTextRepresentation(query)
        #candidats=list({self.index.index_inverse[word].keys() for word in words})
        #scores = { doc: 0 for doc in candidats}
        scores = {}
        for word in words:
            if word in self.index.index_inverse:
                idf = self.index.getIDFForStem(word)
                tfs = self.index.getTfsForStem(word)
                for doc in tfs:
                    val = (idf*tfs[doc])/( tfs[doc] + self.k*( 1 - self.b + self.b*self.lengths[doc]/self.avg_length ))
                    if doc in scores:
                        scores[doc] += val
                    else:
                        scores[doc] = val
        return scores
#     def getRanking(self, query):
#         d = self.getScores(query)
#         sorted_ = sorted(d.items(), key=lambda kv: kv[1], reverse=True)
#         return list(collections.OrderedDict(sorted_).keys())
        
        

In [15]:
indexer=IndexerSimple("cacm.txt")

In [60]:
bm25 = BM25(indexer, 1, 0.75)

In [263]:
stemmer = PorterStemmer()
stemmed_doc = stemmer.getTextRepresentation(indexer.getStrDoc("1502")) 
stemmed_query = stemmer.getTextRepresentation( "What articles exist which deal with TSS (Time Sharing System), anoperating system for IBM computers?")
for x in stemmed_query:
    if x in stemmed_doc:
        print(x, stemmed_doc[x])

In [63]:
class Query():
    def __init__(self, id, query, rel):
        self.id = id
        self.query = query
        self.rel = rel
        
    def get_doc(self, rank):   
        if rank <= len(self.rel) and rank > 0 :
            return self.rel[rank-1]
        
        
class QueryParser():
    def __init__(self, q_file, rel_file):
        self.q_file = q_file
        self.rel_file = rel_file
        
    def parse(self):
        
        with open(self.rel_file, 'r') as f_qrel:
            rels = {}
            while True:
                line = f_qrel.readline()
                matchObj = re.match("([0-9]*) ([0-9]*) .*" , line )
                if matchObj:
                    q_id = str(int(matchObj.group(1)))
                    doc = matchObj.group(2)
                    if q_id in rels :
                        rels[q_id].append(doc)
                    else:
                        rels[q_id] = [doc]
                        
                if line == "":
                    break
                    
        var = open(self.q_file,'r')
        fichier=var.read().replace("\n",'#').replace("\t",' ')
        x = re.findall("(?<=\.I )(\d*.*?)(?=(\.I \d*|\Z))",fichier)
        
        result = []
        
        for elt in x:
            st = elt[0]+elt[1]
            id=re.findall("(\d*?)#\.",elt[0])
            if id[0] in rels:
                y = re.findall("(?<=\.W#)(.*?)(?=\.[A-Z]#)",st)
                y = re.sub("#", " ", y[0])
                result.append(Query(id[0], y, rels[id[0]]))
            
        return result

In [64]:
class EvalMesure():
    def evalQuery(self, docs, query):
        return

class Recip_rank(EvalMesure):
    
    @staticmethod
    def evalQuery(docs, query):  
        first_doc = query.get_doc(1)
        if first_doc in docs:
            return 1/(docs.index(first_doc)+1)
        else:
            return 1/len(docs)

In [166]:
class NDCG_at_k(EvalMesure):
    
    @staticmethod
    def evalQuery(docs, query, k):  
        if k > len(docs):
            k = len(docs)
        idcg = np.array([1/math.log2(i) for i in range(2, len(query.rel)+1)]).sum()+1
        doc = docs[0]
        if doc in query.rel:
            dcg = 1
        else:
            dcg = 0
        for i in range(2, k+1):
            doc = docs[i-1]
            if doc in query.rel:
                dcg += 1/math.log2(i)
        return dcg/idcg
class Precision(EvalMesure):
    @staticmethod
    def evalQuery(docs, query, k):  
        precision=0
        for i in range(k):
            if docs[i] in query.rel:
                precision+=1
        return precision/k
class Recall(EvalMesure):
    @staticmethod
    def evalQuery(docs, query, k):  
        recall=0
        for i in range(k):
            if docs[i] in query.rel:
                recall+=1
        return recall/len(query.rel)
class AvgP(EvalMesure):
    @staticmethod
    def evalQuery(docs, query):  
        div=0
        sum=0
        for i in range(len(docs)):
            if docs[i] in query.rel:
                sum+=Precision.evalQuery(docs,query,i+1)
                div+=1
        if sum==0:
            return 0
        return sum/div
class FMeasure(EvalMesure):
    @staticmethod
    def evalQuery(docs, query, k):  
        precision=Precision.evalQuery(docs,query,k)
        recall=Recall.evalQuery(docs,query,k)
        f_measure= (2*precision*recall)/(precision+recall)
        return f_measure

In [66]:
parser = QueryParser("cacm/cacm.qry", "cacm/cacm.rel")

In [67]:
query_rels = parser.parse()

In [125]:
query = query_rels[0]

docs = bm25.getRanking(query.query)
docs_= lang_model.getRanking(query.query)
Recip_rank.evalQuery(docs, query)
Precision.evalQuery(docs, query, 100)
Precision.evalQuery(docs_,query,100)

0.04

In [126]:
print(NDCG_at_k.evalQuery(docs, query, 100))
print(NDCG_at_k.evalQuery(docs_, query, 100))

0.19423305073221667
0.21161814751112623


In [127]:
print(Recall.evalQuery(docs,query,100))
print(Recall.evalQuery(docs_,query,100))

0.8
0.8


In [128]:
print(AvgP.evalQuery(docs,query))
print(AvgP.evalQuery(docs_,query))

0.04158438666726643
0.050834503842022635


In [167]:
class EvalIRModel():
    
    @staticmethod
    def ndcg(models, queries,  k):
        result = []
        for model in models:
            ndcg = 0
            for query in queries:
                ranking = model.getRanking(query.query)
                ndcg += NDCG_at_k.evalQuery(ranking, query, k)
            ndcg = ndcg/len(queries)
            result.append(ndcg)
        return result
    
    @staticmethod
    def mrr(models, queries):
        result = []
        for model in models:
            mrr = 0
            for query in queries:
                ranking = model.getRanking(query.query)
                mrr += Recip_rank.evalQuery(ranking, query)
            mrr = mrr/len(queries)
            result.append(mrr)
        return result
    @staticmethod
    def precision(models, queries,k):
        result = []
        for model in models:
            precision = 0
            for query in queries:
                ranking = model.getRanking(query.query)
                precision += Precision.evalQuery(ranking, query,k)
            precision = precision/len(queries)
            result.append(precision)
        return result
    @staticmethod
    def recall(models, queries,k):
        result = []
        for model in models:
            recall = 0
            for query in queries:
                ranking = model.getRanking(query.query)
                recall += Recall.evalQuery(ranking, query,k)
            recall = recall/len(queries)
            result.append(recall)
        return result
    @staticmethod
    def map(models, queries):
        result = []
        for model in models:
            map = 0
            for query in queries:
                ranking = model.getRanking(query.query)
                map += AvgP.evalQuery(ranking, query)
            map = map/len(queries)
            result.append(map)
        return result
    @staticmethod
    def f_measure(models, queries,k):
        result = []
        for model in models:
            f_measure = 0
            for query in queries:
                ranking = model.getRanking(query.query)
                f_measure += FMeasure.evalQuery(ranking, query,k)
            f_measure = f_measure/len(queries)
            result.append(f_measure)
        return result

In [326]:
EvalIRModel.ndcg([bm25], query_rels, 4204)

[0.480017840021094]

In [296]:
EvalIRModel.mrr([bm25], query_rels)

[0.09323271242565728]

In [327]:
# test avec sklearn 
arr = np.zeros(4204)
arr_ = np.zeros(4204)
for doc in docs:
    arr[int(doc)-1] = 1
for doc in query.rel:
    arr_[int(doc)-1] = 1
    
    
from sklearn.metrics import ndcg_score
# we have groud-truth relevance of some answers to a query:
true_relevance = np.asarray([arr_])
# we predict some scores (relevance) for the answers
scores = np.asarray([arr])
ndcg_score(true_relevance, scores)

0.18308630091872796

In [222]:
#grid search tme2
train, test = train_test_split( queries, test_size=0.33, random_state=42, shuffle=True)
list_b=np.linspace(0,2,20)
list_k1=np.linspace(0,2,20)
list_landa=np.linspace(0,1,20)

In [223]:
models_bm25=[]
vals=[]
#cpt=0
for b in list_b:
    for k1 in list_k1:
        bm25 = BM25(indexer,k1, b)
        models_bm25.append(bm25)
        t=(b,k1)
        vals.append(t)
#             docs = bm25.getRanking(query)[:20]
#         docs_query.append(docs)
#     docs_bm25[cpt]=docs_query
#     cpt+=1


In [224]:
precisions=np.array(EvalIRModel.precision(models_bm25, train, 100))
map=np.array(EvalIRModel.map(models_bm25, train))

In [225]:
max_precision=np.max(precisions)
best_vals=vals[np.where(precisions==max_precision)[0][0]]
print("meilleures valeurs selon la précision (b,k1)",best_vals," avec une précision de:", max_precision)
max_map=np.max(map)
best_vals=vals[np.where(map==max_map)[0][0]]
print("meilleures valeurs selon map (b,k1)",best_vals," avec une map de:", max_map)

meilleures valeurs selon la précision (b,k1) (1.263157894736842, 1.789473684210526)  avec une précision de: 0.08029411764705882
meilleures valeurs selon map (b,k1) (0.631578947368421, 1.4736842105263157)  avec une map de: 0.35216559977004175


In [226]:
#tester ces valeurs sur les données de test:
map=np.array(EvalIRModel.map([BM25(indexer,best_vals[1], best_vals[0] )], test))
print("map données par les parmametres:",best_vals,"sur les données de test:",map[0])

map données par les parmametres: (0.631578947368421, 1.4736842105263157) sur les données de test: 0.27894587293974094


In [159]:
models_langue=[]
vals=[]
for landa in list_landa:
    langue = ModeleLangue(indexer,landa)
    models_langue.append(langue)
    vals.append(landa)


In [160]:
precisions_l=np.array(EvalIRModel.precision(models_langue, train, 100))
map_l=np.array(EvalIRModel.map(models_langue, train))

In [161]:
max_precision_l=np.max(precisions_l)
best_vals_l=vals[np.where(precisions_l==max_precision_l)[0][0]]
print("meilleures valeurs selon la précision lambda",best_vals_l," avec une précision de:", max_precision_l)
max_map_l=np.max(map_l)
best_vals_l=vals[np.where(map_l==max_map_l)[0][0]]
print("meilleures valeurs selon map lambda",best_vals_l," avec une map de:", max_map_l)

meilleures valeurs selon la précision lambda 0.9473684210526315  avec une précision de: 0.0811764705882353
meilleures valeurs selon map lambda 0.6842105263157894  avec une map de: 0.3847071536954182


In [164]:
#tester ces valeurs sur les données de test:
map=np.array(EvalIRModel.map([ModeleLangue(indexer,best_vals_l)], test))
print("map données par les parmametres:",best_vals_l,"sur les données de test:",map[0])

map données par les parmametres: 0.6842105263157894 sur les données de test: 0.2967698396090046


In [227]:
#cross-validation 
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
list_b=np.linspace(0,1,11)
list_k1=np.linspace(0,2,11)

params_map = []
score_map = []
test_map = []
params_precision = []
score_precision = []
test_precision = []

for train_index, test_index in kf.split(query_rels):
    q_train, q_test = query_rels[train_index], query_rels[test_index]
    models_bm25=[]
    vals=[]
    for b in list_b:
        for k1 in list_k1:
            bm25 = BM25(indexer,k1, b)
            models_bm25.append(bm25)
            t=(b,k1)
            vals.append(t)
            
    prec = np.array(EvalIRModel.precision(models_bm25, q_train, 100))
    mp = np.array(EvalIRModel.map(models_bm25, q_train))
    
    max_precision=np.max(prec)
    best_vals=vals[np.where(prec==max_precision)[0][0]]  
    bm25 = BM25(indexer, best_vals[0], best_vals[1])
    
    params_precision.append(best_vals)
    score_precision.append(max_precision)
    test_precision.append(EvalIRModel.precision([bm25], q_test, 100)[0])
    
    max_map=np.max(mp)
    best_vals=vals[np.where(mp==max_map)[0][0]]  
    bm25 = BM25(indexer, best_vals[0], best_vals[1])
    
    params_map.append(best_vals)
    score_map.append(max_map)
    test_map.append(EvalIRModel.map([bm25], q_test)[0])
    
    
print(params_map)
print(score_map)
print(test_map)
print(params_precision)
print(score_precision)
print(test_precision)

TypeError: only integer scalar arrays can be converted to a scalar index

In [169]:
class HyperlinkIndexer:
    
    def __init__(self,file):
        self.index,self.index_inverse = self.indexation(file)        
        
    def indexation(self,file):
                         
        var = open(file,'r')
        fichier=var.read().replace("\n",'#').replace("\t",' ')
        x = re.findall("(?<=\.I )(\d*.*?)(?=(\.I \d*|\Z))",fichier) 
        index = {}
        index_inv = {}
        
        for elt in x:
            st = elt[0]+elt[1]
            id=re.findall("(\d*?)#\.",elt[0])[0]
            links = re.findall("(?<=\.X#)(.*?)(?=\.(?:I | [A-Z]#))",st)
            if len(links)>0:
                links = links[0].split("#")
                links_ = {}
                for link in links :
                    if link != "" and  re.match("[0-9]* [0-9]* [0-9]*", link) :
                        link_ = link.split()
                        link_ = link_[0]
                        if link_ in links_:
                            links_[link_]+=1
                        else:
                            links_[link_]=1
                            
                        if link_ in index_inv:
                            if id in index_inv[link_]:
                                index_inv[link_][id]+=1
                            else:
                                index_inv[link_][id]=1
                        else:
                            index_inv[link_]={id:1}
                            
                index[id] = links_
        return index, index_inv  
        
    def getHyperlinksTo(self, doc):
        if doc in self.index_inverse:
            return self.index_inverse[doc]

    def getHyperlinksFrom(self, doc):
        if doc in self.index:
            return self.index[doc]

In [203]:
hi = HyperlinkIndexer("cacm.txt")

    
print(hi.getHyperlinksFrom("3230"))

{'1795': 1, '402': 1, '1082': 1, '1181': 1, '373': 1, '2725': 1, '3129': 1}


In [220]:
import random
def sous_graphe(model,n,k, queries, HlinkIndexer):
    result={}
    cpt=1
    for query in queries:
        ranking= model.getRanking(query.query)[:n]
        vq=ranking.copy()
        for doc in ranking:
            vq.extend(list(HlinkIndexer.getHyperlinksFrom(doc).keys()))
            cites=HlinkIndexer.getHyperlinksTo(doc)
            if cites != None:
                if len(cites)>=k:
                    alea=random.sample(list(cites.keys()), k)
                else:
                    alea=random.sample(list(cites.keys()), len(cites))
                vq.extend(alea)
        result[cpt]=list(set(vq))
        cpt+=1
    return result
result=sous_graphe(bm25,20,3,query_rels,hi)       
    

In [221]:
print(result)

{1: ['1991', '2847', '767', '2220', '764', '3119', '2179', '835', '793', '1813', '214', '1245', '994', '2822', '343', '3604', '2879', '2999', '1684', '3230', '183', '738', '428', '402', '3069', '215', '2312', '515', '1352', '1659', '1239', '547', '2873', '2237', '1002', '1194', '361', '442', '2387', '310', '2950', '2156', '1357', '1824', '2665', '2042', '552', '912', '2139', '1108', '372', '759', '1227', '917', '553', '298', '200', '4096', '3140', '1065', '404', '1444', '2343', '3129', '2190', '808', '3489', '255', '3895', '52', '1082', '727', '124', '2988', '855', '2754', '1770', '1950', '2912', '2636', '4139', '446', '3942', '2971', '2274', '2188', '1557', '2418', '961', '705', '1840', '1772', '3020', '2176', '536', '1913', '1945', '40', '1406', '335', '2480', '2862', '1329', '3026', '2572', '355', '2185', '2575', '2753', '1606', '2173', '269', '2622', '3961', '947', '373', '833', '1613', '1053', '2793', '3118', '276', '1795', '2725', '3281', '2776', '195', '2267', '1680', '3194', '1